# GeoBench PASTIS

In [1]:
import os
from pathlib import Path

import torch

from geobench_v2.datamodules import GeoBenchPASTISDataModule
from geobench_v2.datasets import GeoBenchPASTIS
from geobench_v2.datasets.normalization import SatMAENormalizer, ZScoreNormalizer
from geobench_v2.datasets.visualization_util import (
    compute_batch_histograms,
    plot_batch_histograms,
    plot_channel_histograms,
    visualize_segmentation_target_statistics,
)

# %load_ext autoreload
# %autoreload 2

In [2]:
PROJECT_ROOT = Path("/home/nils/projects/GEO-Bench-2/data/pastis")

In [4]:
band_order = GeoBenchPASTIS.band_default_order

datamodule = GeoBenchPASTISDataModule(
    img_size=256,
    batch_size=16,
    num_workers=4,
    # root=PROJECT_ROOT / "data" / "pastis",
    root="/home/nils/projects/GEO-Bench-2/data/pastis",
    band_order=band_order,
    num_time_steps=4,
    data_normalizer=torch.nn.Identity(),  # we do custom normalization in the tutorial
)
datamodule.setup("fit")

Error processing file /home/nils/projects/GEO-Bench-2/data/pastis/geobench_pastis.0000.part.tortilla: Could not open Parquet input source '<Buffer>': Parquet file size is 0 bytes


ValueError: No objects to concatenate

In [ ]:
sample_dist_fig = datamodule.visualize_geospatial_distribution()

### Raw Image Statistics

Computed over the training dataset.

In [ ]:
fig = plot_channel_histograms(PROJECT_ROOT / "pastis_stats_satmae.json")

## Target Statistics

In [ ]:
fig = visualize_segmentation_target_statistics(
    PROJECT_ROOT / "pastis_stats_satmae.json", "PASTIS"
)

### Raw Batch Statistics

In [ ]:
# Get a batch of data from the dataloader
train_dataloader = datamodule.train_dataloader()
raw_batch = next(iter(train_dataloader))

raw_batch_stats = compute_batch_histograms(raw_batch, n_bins=100)


raw_figs = plot_batch_histograms(
    raw_batch_stats, band_order, title_suffix=" (Raw Data)"
)
raw_figs

### Effect of Normalization Schemes

In [ ]:
direct_stats_path = os.path.join(PROJECT_ROOT, "pastis_stats_clip_rescale.json")
satmae_stats_path = os.path.join(PROJECT_ROOT, "pastis_stats_satmae.json")


clip_z_normalizer = ZScoreNormalizer(direct_stats_path, band_order)
satmae_normalizer = SatMAENormalizer(satmae_stats_path, band_order)

### Effect of SatMAE Normalization scheme on batch

In [ ]:
satmae_normalized_batch = satmae_normalizer(raw_batch)
satmae_batch_stats = compute_batch_histograms(satmae_normalized_batch, n_bins=100)
sat_mae_norm_fig = plot_batch_histograms(
    satmae_batch_stats, band_order, title_suffix=" (SatMAE Normalized Data)"
)
sat_mae_norm_fig

### Effect of ClipZ Normalization scheme on batch

In [ ]:
clip_z_normalized_batch = clip_z_normalizer(raw_batch)
clip_z_batch_stats = compute_batch_histograms(clip_z_normalized_batch, n_bins=100)
clip_z_norm_fig = plot_batch_histograms(
    clip_z_batch_stats, band_order, title_suffix=" (Clip Z-Score Normalized Data)"
)
clip_z_norm_fig

## Visualize Batch Data

In [ ]:
fig, batch = datamodule.visualize_batch()